In [9]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re
import os

# 유튜브 크롤링 함수 생성

In [15]:
# 유튜브의 titles와 url을 반환하는 함수
def get_urls_from_youtube_with_keyword(keyword):
    titles = []
    urls = []
    
    search_keyword_encode = requests.utils.quote(keyword)
    
    url = "https://www.youtube.com/results?search_query=" + search_keyword_encode
    
    driver = wd.Chrome(executable_path="./chromedriver_win32/chromedriver.exe") 
    
    driver.get(url)
    
    # 필터 버튼 클릭
    driver.find_element_by_xpath('//*[@id="container"]/ytd-toggle-button-renderer/a').click()

    # 필터 기준 - 업로드 날짜 - 이번 달
    upload_date = driver.find_element_by_xpath('//*[@id="collapse-content"]/ytd-search-filter-group-renderer[1]')
    click_list_1 = upload_date.find_elements_by_id("endpoint")
    click_list_1[3].click()
    
    # 필터 창이 내려가는 속도에 맞추기 위해서 임의로 sleep한 후에 버튼 클릭
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="container"]/ytd-toggle-button-renderer/a').click()

    # 필터 기준 - 정렬 기준 - 조회수
    sorting_std = driver.find_element_by_xpath('//*[@id="collapse-content"]/ytd-search-filter-group-renderer[5]')
    click_list_5 = sorting_std.find_elements_by_id("endpoint")
    click_list_5[1].click()
    time.sleep(2)

    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        
        time.sleep(3.0)
        
        
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        if new_page_height == last_page_height:
            break
            
        last_page_height = new_page_height
        
    html_source = driver.page_source
    
    driver.quit()
    
    soup = BeautifulSoup(html_source, 'lxml')
    
    datas = soup.select("a#video-title")

    for data in datas:
        title = data.text.replace('\n', '')
        url = "https://www.youtube.com/" + data.get('href')
        
        titles.append(title)
        urls.append(url)
        
    return titles, urls

In [16]:
# urls 입력받아 html_sources를 반환하는 함수
def crawl_youtube_page_html_sources(urls):
    html_sources = []

    for i in range(1): #len(urls)
        driver = wd.Chrome(executable_path="./chromedriver_win32/chromedriver.exe") 
        driver.get(urls[i])

        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(3.0)
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

            if new_page_height == last_page_height:
                break
            last_page_height = new_page_height


        html_source = driver.page_source
        html_sources.append(html_source)
        print("OK")

        driver.quit()
    return html_sources

In [17]:
# 동영상 제목, 동영상 업로드일, 댓글 작성자, 댓글 내용, 댓글 작성일 크롤링하여 dataframe형식으로 반환하는 함수
def get_user_IDs_and_comments(html_sources):
    my_dataframes = []
    cnt = 0
    for html in html_sources:
        soup = BeautifulSoup(html, 'lxml')
        
        youtube_title = soup.select('#container > h1 > yt-formatted-string')
        
        youtube_title_date = soup.select('#date > yt-formatted-string')
        
        youtube_user_IDs = soup.select('#author-text > span')

        youtube_comments = soup.select('#content-text')
        
        youtube_date = soup.select('#header-author > yt-formatted-string > a')
                
        youtube_channel = soup.select('#text > a')
        
        youtube_like = soup.select('#text')
        
        youtube_dislike = soup.select('#text')
        
        youtube_comments_like = soup.select('#vote-count-middle')
        # print(youtube_comments_like)
        
        str_youtube_title = []
        str_youtube_title_date = []
        str_youtube_userIDs = []
        str_youtube_comments = []
        str_youtube_date = []
        str_youtube_channel = []
        str_youtube_like = []
        str_youtube_dislike = []
        str_youtube_comments_like = []
        
        cnt += 1 
        print(cnt)
        
        for i in range(len(youtube_user_IDs)):
            str_youtube_title.append(str(youtube_title[0].text))
            
            str_youtube_title_date.append(str(youtube_title_date[0].text))
            
            str_youtube_channel.append(str(youtube_channel[0].text))
            
            str_youtube_like.append(str(youtube_like[2].text))
            
            str_youtube_dislike.append(str(youtube_dislike[3].text))
            
            str_tmp = str(youtube_user_IDs[i].text)
            # print(str_tmp)
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('                ','')
            str_tmp = str_tmp.strip()
            str_youtube_userIDs.append(str_tmp)

            str_tmp = str(youtube_comments[i].text) 
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('               ', '')
            str_youtube_comments.append(str_tmp)
            
            str_tmp = str(youtube_date[i].text) 
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('               ', '')
            str_youtube_date.append(str_tmp)
            
            str_tmp = str(youtube_comments_like[i].text)
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('  ', '')
            str_youtube_comments_like.append(str_tmp)
            print(str_tmp)
        
        pd_data = {"Title" : str_youtube_title,
                   "Date" : str_youtube_title_date,
                   "ID" : str_youtube_userIDs, 
                   "Comment" : str_youtube_comments, 
                   "Comment_date" : str_youtube_date,
                   "Channel" : str_youtube_channel,
                   "Like" : str_youtube_like,
                   "Dislike" : str_youtube_dislike,
                   "Comment_like" : str_youtube_comments_like}
        

        youtube_pd = pd.DataFrame(pd_data)

        my_dataframes.append(youtube_pd)
        
    return my_dataframes

In [18]:
# dataframe을 csv파일로 저장하는 함수
def convert_csv_from_dataframe(titles, my_dataframes):
    for i in range(len(my_dataframes)):
        title = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…《\》]', '', titles[i])
        my_dataframes[i].to_csv("data_each/{}.csv".format(title), encoding='utf-8-sig', index = False)        

In [19]:
titles, url = get_urls_from_youtube_with_keyword("CFA")

html_sources = crawl_youtube_page_html_sources(url)

my_dataframes = get_user_IDs_and_comments(html_sources)

convert_csv_from_dataframe(titles, my_dataframes)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=90.0.4430.93)


# 개별 csv파일을 모두 이어 붙인 DataFrame 생성

In [7]:
path ='../data/유튜브수집데이터/data_each'
file_list = os.listdir(path)

In [8]:
total = pd.read_csv(path +  '/' + file_list[0])
for i in range(len(file_list)) :
        total =  pd.concat([total,pd.read_csv(path +  '/' + file_list[i])])

In [9]:
total

,Title,Date,ID,Comment,Comment_date,Channel,Like,Dislike,Comment_like
0,"2030표심, 2022대선전략 : 여성가족부 폐지, 여성징병제 = 양성징병제 시행",2021. 4. 21.,성수,진짜 저 공약들 들고나오면 두말안하고 바로 그냥 새벽6시 되자마자 투표하러간다,1일 전,게임농부,16,0,6
1,"2030표심, 2022대선전략 : 여성가족부 폐지, 여성징병제 = 양성징병제 시행",2021. 4. 21.,Girl Fighter,여성가족부 여성단체 북한 . 여성가족부와 여성단체의 심각성은 이제 북한급입니다.,1일 전,게임농부,16,0,3
2,"2030표심, 2022대선전략 : 여성가족부 폐지, 여성징병제 = 양성징병제 시행",2021. 4. 21.,Magic Vilon,일본여자=의무를 안하지만 권리도 포기함미국여자=의무도 전부 수행하고 권리도 챙김한국...,1일 전,게임농부,16,0,4
3,"2030표심, 2022대선전략 : 여성가족부 폐지, 여성징병제 = 양성징병제 시행",2021. 4. 21.,mama papa,여가부가 폐지되려면본인들을 우파라고 하는 정치인들이 여성친화같은 페미니즘 정책이 사...,1일 전,게임농부,16,0,1
4,"2030표심, 2022대선전략 : 여성가족부 폐지, 여성징병제 = 양성징병제 시행",2021. 4. 21.,TV팽타,여성 징병은 당연히 실행되어야 하는 것이고 추후 모병제로 변경될 경우에는 모병제 이...,1일 전,게임농부,16,0,1
5,"2030표심, 2022대선전략 : 여성가족부 폐지, 여성징병제 = 양성징병제 시행",2021. 4. 21.,미국,2017년 홍준표 공약중 - 여성가족부 폐지하고 청년담당부처로 전환하고 규모 축소하...,10분 전,게임농부,16,0,0
0,"2030표심, 2022대선전략 : 여성가족부 폐지, 여성징병제 = 양성징병제 시행",2021. 4. 21.,성수,진짜 저 공약들 들고나오면 두말안하고 바로 그냥 새벽6시 되자마자 투표하러간다,1일 전,게임농부,16,0,6
1,"2030표심, 2022대선전략 : 여성가족부 폐지, 여성징병제 = 양성징병제 시행",2021. 4. 21.,Girl Fighter,여성가족부 여성단체 북한 . 여성가족부와 여성단체의 심각성은 이제 북한급입니다.,1일 전,게임농부,16,0,3
2,"2030표심, 2022대선전략 : 여성가족부 폐지, 여성징병제 = 양성징병제 시행",2021. 4. 21.,Magic Vilon,일본여자=의무를 안하지만 권리도 포기함미국여자=의무도 전부 수행하고 권리도 챙김한국...,1일 전,게임농부,16,0,4
3,"2030표심, 2022대선전략 : 여성가족부 폐지, 여성징병제 = 양성징병제 시행",2021. 4. 21.,mama papa,여가부가 폐지되려면본인들을 우파라고 하는 정치인들이 여성친화같은 페미니즘 정책이 사...,1일 전,게임농부,16,0,1


# csv파일 저장

In [10]:
total.to_csv('data/2030표심.csv', encoding='utf-8-sig')